# T5rus с промптом + LoRA (через псевдотекст)

Будем генерировать не json по сцене а псевдотекст, также T5(Text-To-Text Transfer Transformer) + LoRA

In [1]:
import json
import torch
import os 
import sys
import warnings
import re

import numpy as np

from pathlib import Path
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import TrainerCallback
from torch.utils.data import DataLoader

from peft import LoraConfig, get_peft_model, TaskType, PeftConfig,PeftModel
from tqdm import tqdm

import matplotlib.pyplot as plt

# отключаем их все чтобы картинку не портили
warnings.filterwarnings("ignore", category=FutureWarning)


#DATA_DIR = Path("../dataset/dataset_syntetic_v4").expanduser()
DATA_DIR = Path("../dataset/dataset_tmp").expanduser()
MODEL_NAME = "sberbank-ai/ruT5-base"

lib_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(lib_path)

from library.metrics_pseudo import evaluate_scene_extraction, evaluate_global_f1_on_pairs
from library.safe_compute_metrics import safe_compute_metrics

/home/sda/anaconda3/envs/tenv/lib/python3.12/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/sda/anaconda3/envs/tenv/lib/python3.12/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


### Перевод в псевдокод/псевдотекст и обратно

Чтобы не заставлять модель выучивать корректный json сделаем промежуточный перевод в псевдокод

In [18]:
import re

def json_to_pseudo_text(json_data):
    """
    Преобразует список объектов с признаками из JSON в псевдоформат:
    название | признак1, признак2
    Если признаков нет — просто "название |"
    """
    lines = []
    for obj_dict in json_data:
        for obj_name, attrs in obj_dict.items():
            attr_text = ", ".join(attrs) if attrs else ""
            line = f"{obj_name} | {attr_text}"
            lines.append(line)
    return "\n".join(lines)

def pseudo_text_to_json(text):
    """
    Обратное преобразование псевдоформата в список объектов с признаками.
    """
    result = []
    lines = text.strip().split("\n")
    for line in lines:
        match = re.match(r"\s*(.+?)\s*\|\s*(.*)", line.strip())
        if match:
            obj_name = match.group(1).strip()
            attrs_str = match.group(2).strip()
            attrs = [attr.strip() for attr in attrs_str.split(",")] if attrs_str else []
            result.append({obj_name: attrs})
    return result


In [19]:
original = [{"домкрат": ["металлический", "тяжелый"]}, {"аккумулятор": []}]
pseudo = json_to_pseudo_text(original)
print(pseudo)
#recovered = pseudo_text_to_json(pseudo)
#assert original == recovered

print(recovered)

домкрат | металлический, тяжелый
аккумулятор | 
[{'гаечный ключ': ['металлический']}, {'домкрат': ['металлический', 'тяжелый', 'прочный']}, {'аккумулятор': []}]


In [21]:
pseudo = """
гаечный ключ |  металлический  
домкрат |    металлический, тяжелый, прочный  
аккумулятор |  
"""

recovered = pseudo_text_to_json(pseudo)
print(recovered)

[{'гаечный ключ': ['металлический']}, {'домкрат': ['металлический', 'тяжелый', 'прочный']}, {'аккумулятор': []}]


In [5]:
PROMPT = """Описание сцены: {description}

Инструкция: выдели все объекты и их признаки в следующем формате.

Формат:
<название объекта> | <список признаков через запятую>

Если у объекта нет признаков, напиши "-".

Ответь только в указанном формате без лишнего текста.
"""

In [6]:
class CustomEvaluateCallback(TrainerCallback):
    def __init__(self, val_dataset, tokenizer, eval_fn, save_path="./metrics_v2/"):
        super().__init__()
        self.val_dataset = val_dataset
        self.tokenizer = tokenizer
        self.eval_fn = eval_fn
        self.save_path = Path(save_path)
        self.save_path.mkdir(exist_ok=True, parents=True)
        self.metrics_history = []

    def on_epoch_end(self, args, state, control, **kwargs):
        model = kwargs["model"]
        device = model.device

        print(f"\nCustom evaluation starting for epoch {state.epoch:.2f}...")

        model.eval()

        all_preds = []
        all_labels = []

        for example in self.val_dataset:
            # готовим инпут
            prompt = PROMPT.format(description=example["description"])
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)

            with torch.no_grad():
                output_ids = model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    max_length=256,
                    num_beams=4,
                    early_stopping=True
                )

            output_text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

            try:
                pred_json = pseudo_text_to_json(output_text)
            except Exception as e:
                print(f"Ошибка парсинга предсказания: {e}")
                pred_json = {}

            try:
                gt_json = example["scene_objects"]
            except Exception as e:
                print(f"Ошибка получения ground truth: {e}")
                gt_json = {}

            scores = self.eval_fn(gt_json, pred_json)

            all_preds.append(scores)

        if not all_preds:
            print("Нет валидных примеров для оценки.")
            return control

        avg_scores = {
            "epoch": state.epoch,
            "f1_object": sum(s["f1_object"] for s in all_preds) / len(all_preds),
            "f1_attribute": sum(s["f1_attribute"] for s in all_preds) / len(all_preds),
            "f1_combined_weighted": sum(s["f1_combined_weighted"] for s in all_preds) / len(all_preds),
            "f1_combined_simple": sum(s["f1_combined_simple"] for s in all_preds) / len(all_preds),
        }

        self.metrics_history.append(avg_scores)

        print(f"Epoch {state.epoch:.2f} scores:", avg_scores)

        # Сохраняем историю
        with open(self.save_path / "metrics_history.json", "w", encoding="utf-8") as f:
            json.dump(self.metrics_history, f, indent=2, ensure_ascii=False)

        # Строим графики
        self.plot_metrics()

        return control

    def plot_metrics(self):
        if not self.metrics_history:
            return

        epochs = [m["epoch"] for m in self.metrics_history]

        plt.figure(figsize=(10, 6))
        for key in ["f1_object", "f1_attribute", "f1_combined_weighted", "f1_combined_simple"]:
            plt.plot(epochs, [m[key] for m in self.metrics_history], label=key)

        plt.xlabel("Эпоха")
        plt.ylabel("F1 Score")
        plt.title("F1 по эпохам")
        plt.legend()
        plt.grid(True)
        plt.savefig(self.save_path / "learning_curves.png")
        plt.close()


In [7]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def safe_parse_json(text):
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return None

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Если predictions — logits, нужно брать argmax

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # logits -> ids
    predictions = np.argmax(predictions, axis=-1)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print("preds:", decoded_preds[0])    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    f1_object_list = []
    f1_attribute_list = []
    f1_combined_weighted_list = []
    f1_combined_simple_list = []
    valid = 0

    for pred_text, label_text in zip(decoded_preds, decoded_labels):
        pred_json = safe_parse_json(pred_text)
        label_json = safe_parse_json(label_text)

        if pred_json is None or label_json is None:
            # Невалидный JSON → 0 по всем метрикам
            f1_object_list.append(0.0)
            f1_attribute_list.append(0.0)
            f1_combined_weighted_list.append(0.0)
            f1_combined_simple_list.append(0.0)
        else:
            valid += 1
            scores = evaluate_scene_extraction(label_json, pred_json)
            f1_object_list.append(scores["f1_object"])
            f1_attribute_list.append(scores["f1_attribute"])
            f1_combined_weighted_list.append(scores["f1_combined_weighted"])
            f1_combined_simple_list.append(scores["f1_combined_simple"])

    total = len(decoded_preds)

    return {
        "f1_object": round(sum(f1_object_list) / total, 4),
        "f1_attribute": round(sum(f1_attribute_list) / total, 4),
        "f1_combined_weighted": round(sum(f1_combined_weighted_list) / total, 4),
        "f1_combined_simple": round(sum(f1_combined_simple_list) / total, 4),
        "valid_json_rate": round(valid / total, 4),
        "total_samples": total,
        "valid_samples": valid,
    }

In [8]:
data = []
for path in sorted(DATA_DIR.glob("*.jsonl")):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            description = item["description"]
            scene_objects = item["scene"]["objects"]
            data.append({
                "description": description,
                "scene_objects": scene_objects
            })


dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_ds, val_ds = dataset["train"], dataset["test"]

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def preprocess(example):
    prompt = PROMPT.format(description=example["description"])
    inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)

    pseudo_target = json_to_pseudo_text(example["scene_objects"])
    targets = tokenizer(pseudo_target, padding="max_length", truncation=True, max_length=256)

    inputs["labels"] = targets["input_ids"]
    return inputs


train_ds = train_ds.map(preprocess, batched=False)
val_ds = val_ds.map(preprocess, batched=False)

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

lora_config = LoraConfig(
    r=8, # ранг низкоранговой матрицы
    lora_alpha=16,
    target_modules=["q", "v"],  # ruT5 может требовать уточнения слоёв (или просто "SelfAttention")
    # target_modules=["q", "k", "v", "o"]
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)

# масштабируем результат через lora_alpha / r, чтобы управлять "силой" влияния LoRA на исходный поток данных.
training_args = TrainingArguments(
    output_dir="./rut5_lora_outputs_v2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=20,
    logging_dir="./logs_v2",
    logging_steps=50,
    evaluation_strategy="epoch",
    eval_accumulation_steps=1, # для маленькой памяти GPU   
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
    fp16=True  
)

custom_callback = CustomEvaluateCallback(
    val_dataset=val_ds,
    tokenizer=tokenizer,
    eval_fn=evaluate_scene_extraction,
    save_path="./metrics_v2/"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
    callbacks=[custom_callback]
)


#trainer.train(resume_from_checkpoint=True)
trainer.train()
model.save_pretrained("./rut5_lora_outputs_v2")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/4801 [00:00<?, ? examples/s]

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,F1 Object,F1 Attribute,F1 Combined Weighted,F1 Combined Simple,Valid Json Rate,Total Samples,Valid Samples
1,0.229500,0.106075,0.000000,0.000000,0.000000,0.000000,0.000000,253,0


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🔵 Custom evaluation starting for epoch 1.00...
✅ Epoch 1.00 scores: {'epoch': 1.0, 'f1_object': 0.0, 'f1_attribute': 0.0, 'f1_combined_weighted': 0.0, 'f1_combined_simple': 0.0}
preds: объект: автомобиль  признаки: - объект: аккумулятор  признаки: - объект: алтарь  признаки: - объект: бар  признаки: - объект: бар  признаки: - объект: барная_стойка  признаки: - объект: барный стул  признаки: - объект: билет  признаки: - объект: билет  признаки: - объект: бокал  признаки: - объект: ботинки  признаки: - объект: ботинок  признаки: - объект: бутылка  признаки: - объект: вагон  признаки: - объект: ведроаза  признаки: - объект: ведро  признаки: - объект: вентиленик  признаки: - объект: вентиль  признаки: - объект: верстак  признаки: - объект: весы  признаки: - объект: вешалка  признаки: - объект: вешалилка  признаки: - объект: витрина  признаки: - объект: волна  признаки: - объект: воротаольер  признаки: - объект: ворота  признаки: - объект: ечный ключ  признаки: - объект: гантель  признаки:

/home/sda/anaconda3/envs/tenv/lib/python3.12/site-packages/peft/tuners/lora.py:297: SyntaxWarning: invalid escape sequence '\.'
  re.match(f".*\.{target_key}$", key) for target_key in lora_config.target_modules
/home/sda/anaconda3/envs/tenv/lib/python3.12/site-packages/peft/tuners/lora.py:307: SyntaxWarning: invalid escape sequence '\.'
  layer_index = re.match(f".*.{pattern}\.(\d+)\.*", key)


KeyboardInterrupt: 

### Проверка

In [ ]:
MODEL_DIR = "./rut5_lora_outputs"  # путь к fine-tuned модели
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# === 1. Загрузка модели и токенизатора ===
print("Loading model...")
config = PeftConfig.from_pretrained(MODEL_DIR)
base_model = T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, MODEL_DIR)
model = model.to(DEVICE)
model.eval()

tokenizer = T5Tokenizer.from_pretrained(config.base_model_name_or_path)

# === 2. Генерация ===
def predict(description: str, max_length: int = 256):
    prompt = PROMPT.format(description=description)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(DEVICE)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    try:
        parsed_json = json.loads(output_text)
    except Exception as e:
        print(f"Ошибка парсинга JSON: {e}")
        print("Сырые данные:", output_text)
        parsed_json = None

    return parsed_json


# === 3. Пример использования ===
if __name__ == "__main__":
    text = input("Введите описание сцены: ")
    result = predict(text)
    print("\nПредсказание:\n")
    print(json.dumps(result, indent=2, ensure_ascii=False))


In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    total_params = 0

    for param in model.parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"Всего параметров: {total_params / 1e6:.2f}M")
    print(f"Обучаемых параметров: {trainable_params / 1e6:.2f}M")
    print(f"Доля обучаемых параметров: {100 * trainable_params / total_params:.2f}%")

# Вызов функции после создания модели

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)